In [57]:
# Batch GD
class GDregressor:
    def __init__(self, learning_rate=0.01, epochs=100):
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.coef_ = None
        self.intercept_ = None

    def fit(self, X_train, y_train):
        X_train = np.array(X_train)
        y_train = np.array(y_train).reshape(-1,1)

        m, n = X_train.shape

        # initialize
        self.intercept_ = 0.0
        self.coef_ = np.ones((n,1))

        for _ in range(self.epochs):

            # prediction
            # vectorization
            y_hat = (X_train @ self.coef_) + self.intercept_

            # error
            error = y_train - y_hat

            # gradients
            d_b = -2 * np.mean(error)
            d_w = -2 * (X_train.T @ error) / m

            # update
            self.intercept_ -= self.learning_rate * d_b
            self.coef_ -= self.learning_rate * d_w

            loss = np.mean(error**2)

            if _ % 300 == 0:
                print("Epoch:", _, "Loss:", loss)


    def predict(self, X_test):
        X_test = np.array(X_test)
        return X_test @ self.coef_ + self.intercept_

    def mse(self, y_true, y_pred):
        return np.mean((y_true - y_pred)**2)

    def r2_score(self,y_true, y_pred):
        y_true = np.array(y_true).reshape(-1,1)
        y_pred = np.array(y_pred).reshape(-1,1)

        ss_res = np.sum((y_true - y_pred) ** 2)   # residual sum of squares
        ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)  # total sum of squares

        r2 = 1 - (ss_res / ss_tot)
        return r2

# Stochastic GD
class SGDregressor:
    def __init__(self, learning_rate=0.01, epochs=100):
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.coef_ = None
        self.intercept_ = None

    def fit(self, X_train, y_train):
        X_train = np.array(X_train)
        y_train = np.array(y_train).reshape(-1,1)

        m, n = X_train.shape

        # initialize
        self.intercept_ = 0.0
        self.coef_ = np.ones((n,1))

        for epoch in range(self.epochs):
            for i in range(X_train.shape[0]):
                idx = np.random.randint(0, X_train.shape[0])

                xi = X_train[idx].reshape(1, -1)
                yi = y_train[idx]

                # prediction (single sample)
                # vectorization
                y_hat = (xi @ self.coef_) + self.intercept_
    
                # error (single sample)
                error = yi - y_hat
    
                # gradients (single sample)
                d_b = -2 * error
                d_w = -2 * (xi.T @ error)
    
                # update immediately
                self.intercept_ -= self.learning_rate * d_b
                self.coef_ -= self.learning_rate * d_w
    
    
            # monitoring loss 
            if epoch % 200 == 0:
                y_full_pred = X_train @ self.coef_ + self.intercept_
                loss = np.mean((y_train - y_full_pred)**2)
                print("Epoch:", epoch, "Loss:", loss)


    def predict(self, X_test):
        X_test = np.array(X_test)
        return X_test @ self.coef_ + self.intercept_

    def mse(self, y_true, y_pred):
        return np.mean((y_true - y_pred)**2)

    def r2_score(self,y_true, y_pred):
        y_true = np.array(y_true).reshape(-1,1)
        y_pred = np.array(y_pred).reshape(-1,1)

        ss_res = np.sum((y_true - y_pred) ** 2)   # residual sum of squares
        ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)  # total sum of squares

        r2 = 1 - (ss_res / ss_tot)
        return r2

# Mini Batch GD
class MiniBatchGDregressor:
    def __init__(self, batch_size, learning_rate=0.01, epochs=100):
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.coef_ = None
        self.intercept_ = None
        self.batch_size = batch_size

    def fit(self, X_train, y_train):
        X_train = np.array(X_train)
        y_train = np.array(y_train).reshape(-1,1)

        m, n = X_train.shape
        batch = m // self.batch_size

        # initialize
        self.intercept_ = 0.0
        self.coef_ = np.ones((n,1))

        for epoch in range(self.epochs):
            for i in range(batch):
                idx = np.random.choice(m, self.batch_size, replace=False)

                xi = X_train[idx]
                yi = y_train[idx]

                # prediction 
                # vectorization
                y_hat = (xi @ self.coef_) + self.intercept_
    
                # error 
                error = yi - y_hat
    
                # gradients 
                d_b = -2 * np.mean(error)
                d_w = -2 * (xi.T @ error) / self.batch_size
    
                # update immediately
                self.intercept_ -= self.learning_rate * d_b
                self.coef_ -= self.learning_rate * d_w
    
    
            # monitoring loss 
            if epoch % 200 == 0:
                y_full_pred = X_train @ self.coef_ + self.intercept_
                loss = np.mean((y_train - y_full_pred)**2)
                print("Epoch:", epoch, "Loss:", loss)


    def predict(self, X_test):
        X_test = np.array(X_test)
        return X_test @ self.coef_ + self.intercept_

    def mse(self, y_true, y_pred):
        return np.mean((y_true - y_pred)**2)

    def r2_score(self,y_true, y_pred):
        y_true = np.array(y_true).reshape(-1,1)
        y_pred = np.array(y_pred).reshape(-1,1)

        ss_res = np.sum((y_true - y_pred) ** 2)   # residual sum of squares
        ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)  # total sum of squares

        r2 = 1 - (ss_res / ss_tot)
        return r2



In [58]:
import numpy as np

# function to generate a raw data
def generate_lr_data(
    n_samples=200,
    n_features=1,
    noise_std=1.0,
    weight_range=(-5, 5),
    bias_range=(-3, 3),
    x_range=(-10, 10),
    add_outliers=False,
    outlier_ratio=0.05,
    outlier_strength=15,
    seed=42
):
    """
    Generate synthetic Linear Regression data:
        y = Xw + b + noise

    Returns:
        X : (n_samples, n_features)
        y : (n_samples, 1)
        true_w : (n_features, 1)
        true_b : float
    """
    rng = np.random.default_rng(seed)

    # Features
    X = rng.uniform(x_range[0], x_range[1], size=(n_samples, n_features))

    # True weights and bias
    true_w = rng.uniform(weight_range[0], weight_range[1], size=(n_features, 1))
    true_b = rng.uniform(bias_range[0], bias_range[1])

    # Perfect line/plane
    y_clean = X @ true_w + true_b

    # Add Gaussian noise
    noise = rng.normal(0, noise_std, size=(n_samples, 1))
    y = y_clean + noise

    # Optionally add outliers
    if add_outliers:
        n_outliers = int(n_samples * outlier_ratio)
        outlier_indices = rng.choice(n_samples, n_outliers, replace=False)

        # Make y much bigger/smaller randomly
        y[outlier_indices] += rng.normal(0, outlier_strength, size=(n_outliers, 1))

    return X, y, true_w, true_b


def train_test_split_manual(X, Y, test_size=0.2, random_state=42):
    rng = np.random.default_rng(random_state)

    X = np.array(X)
    Y = np.array(Y).reshape(-1, 1)

    n_samples = X.shape[0]

    indices = np.arange(n_samples)
    rng.shuffle(indices)

    test_count = int(n_samples * test_size)

    test_idx = indices[:test_count]
    train_idx = indices[test_count:]

    return X[train_idx], X[test_idx], Y[train_idx], Y[test_idx]


In [59]:
X, y, w_true, b_true = generate_lr_data(
    n_samples=300,
    n_features=10,
    noise_std=1.5
)

print("True weights:", w_true.ravel())
print("True bias:", b_true)


True weights: [ 3.29221141  0.94095987 -2.55145253  2.45675001 -4.15519104  2.74417834
  0.89985222 -3.89249682 -4.20494437  2.50836333]
True bias: -1.242384560976977


In [60]:
X_train, X_test, y_train, y_test = train_test_split_manual(X, y, test_size=0.2, random_state=2)

"""
    During training, gradient descent initially diverged due to large feature magnitudes.
    This was resolved by applying z-score normalization (mean=0, std=1) to input features, which significantly improved convergence stability.
"""
mean = X_train.mean(axis=0)
std = X_train.std(axis=0)

X_train = (X_train - mean) / std
X_test = (X_test - mean) / std



gdr = GDregressor(learning_rate=0.001, epochs=1000)
gdr.fit(X_train, y_train)

print("True:", w_true.ravel(), b_true)
print("Learned:", gdr.coef_.ravel(), gdr.intercept_)


# My gradient descent converged successfully as indicated by decreasing loss; 
# coefficient values differ from ground truth due to feature standardization.

Epoch: 0 Loss: 2950.5457335678684
Epoch: 300 Loss: 895.9227510035776
Epoch: 600 Loss: 284.0153149452275
Epoch: 900 Loss: 94.10280852559185
True: [ 3.29221141  0.94095987 -2.55145253  2.45675001 -4.15519104  2.74417834
  0.89985222 -3.89249682 -4.20494437  2.50836333] -1.242384560976977
Learned: [ 18.46099573   4.97743551 -13.39405846  11.64592431 -20.81560901
  12.28338385   3.84527283 -17.71291302 -19.9999052   12.57180844] 3.082166908407099


In [61]:
sgdr = SGDregressor(learning_rate=0.00001, epochs=1000)
sgdr.fit(X_train, y_train)

print("Learned:", sgdr.coef_.ravel(), sgdr.intercept_)

Epoch: 0 Loss: 2918.8928011844487
Epoch: 200 Loss: 443.77816944221865
Epoch: 400 Loss: 74.8567149698908
Epoch: 600 Loss: 14.8797086907648
Epoch: 800 Loss: 4.430082253176249
Learned: [ 19.67275196   5.44991575 -14.93405384  14.0527605  -24.07134559
  15.15189888   4.98979917 -21.42909965 -23.72043843  13.96401134] [[3.52723066]]


In [63]:
mbgdr = MiniBatchGDregressor(batch_size=10, learning_rate=0.001, epochs=1000)
mbgdr.fit(X_train, y_train)

print("Learned:", mbgdr.coef_.ravel(), mbgdr.intercept_)

Epoch: 0 Loss: 2676.487726717172
Epoch: 200 Loss: 1.9957800589386727
Epoch: 400 Loss: 1.9950972511567073
Epoch: 600 Loss: 1.9957522885074355
Epoch: 800 Loss: 1.9968757789631004
Learned: [ 19.59415269   5.45796486 -15.03821146  14.36902377 -24.3859084
  15.45489712   5.10671334 -21.76287649 -24.05845986  14.05925977] 3.557302054736163


In [52]:
y_pred = gdr.predict(X_test)

print("Predicted:", y_pred.ravel())

Predicted: [  7.87507253  43.65022759 116.33122892 -97.61044568  10.98960873
 -66.55188456  -6.00581183 -75.21054312 -64.08465188  21.85072646
  -1.3742951   18.90881861 -20.23757512 -39.14022115  14.44685565
  55.04129086 -13.53423353  35.66211641  56.11425419  33.71423522
  10.28499852  36.58364661  41.83397078 -55.51472599 -41.52125557
  -7.10941503 -48.74700215 -47.45037428 -35.98452663  -0.73011202
  62.33264493 -35.53188691 -42.57024925  94.50073939  59.14680544
 -77.64666077  -2.68067512  22.84375457 -23.00238024  60.35826543
 -12.10265045  60.0931546   30.15824817  52.50697398 -44.15710453
 -58.57910068  43.52024377 -43.67416157  53.42677804 107.11559797
 -77.06189641 -33.91890756  39.61278986  -1.90167891  22.48187406
  16.29033505  -4.52981196  26.96054541  25.94135286 -91.45381713]


In [53]:
y_pred_sgdr = sgdr.predict(X_test)

print("Predicted:", y_pred_sgdr.ravel())

Predicted: [   5.27507651   52.6218521   134.06636494 -113.47682491   14.36273915
  -78.72958039   -2.12135383  -85.49701391  -77.43893754   26.10915491
   -0.89017354   24.86687535  -21.78424449  -44.11614203   10.34345156
   65.08919613  -19.91541079   38.70756746   65.19115567   40.72570536
   10.13752059   42.20917729   49.50280623  -63.88193217  -54.9503731
   -8.43561331  -53.70659603  -58.95443383  -40.45918228   -2.37622168
   72.96265173  -45.54746749  -47.37497479  106.83934682   65.27957841
  -90.58119313   -2.80380982   23.03666296  -21.92173446   65.59543882
  -12.43331415   67.72170434   36.38284347   57.26157082  -50.15653934
  -66.18134555   50.01847041  -46.25710508   62.32554234  122.75473067
  -88.90460949  -34.08129465   47.7890786     0.34805422   30.12979135
   18.6060617    -6.43909762   33.53286359   27.43059393 -106.66747322]


In [64]:
y_pred_mbgdr = mbgdr.predict(X_test)

print("Predicted:", y_pred_mbgdr.ravel())

Predicted: [   4.82959225   53.60171421  135.37379013 -114.90887754   14.59847164
  -79.78618171   -1.4292781   -86.20163104  -78.8943219    26.5359844
   -1.07189789   25.54186794  -21.8263538   -44.38437716    9.47674972
   66.0032016   -20.74163011   38.68720934   65.73897147   41.49739227
    9.91395833   42.65755799   50.3005705   -64.33247152  -56.75328935
   -8.42209429  -53.95154348  -60.37996864  -40.7870818    -2.79184397
   73.86603853  -46.79430713  -47.77535616  107.71207952   65.45146022
  -91.7984467    -2.86252581   22.69398081  -21.45438343   65.74138017
  -12.40795017   68.30563982   36.85162656   57.36853416  -50.63643166
  -66.54050496   50.43403117  -45.9698451    62.8778155   124.01894693
  -89.78980115  -33.70056273   48.89505948    0.65936222   31.15973364
   18.50975696   -6.73533004   34.32087548   27.42412049 -107.91989061]


In [54]:
mse = gdr.mse(y_test, y_pred)
r2 = gdr.r2_score(y_test, y_pred)

print("MSE:", mse)
print("R2 Score:", r2)

MSE: 77.95026515026714
R2 Score: 0.9762891612287307


In [56]:
mse_sgdr = sgdr.mse(y_test, y_pred_sgdr)
r2_sgdr = sgdr.r2_score(y_test, y_pred_sgdr)

print("MSE:", mse_sgdr)
print("R2 Score:", r2_sgdr)

MSE: 2.92362716781586
R2 Score: 0.9991106938216342


In [65]:
mse_mbgdr = mbgdr.mse(y_test, y_pred_mbgdr)
r2_mbgdr = mbgdr.r2_score(y_test, y_pred_mbgdr)

print("MSE:", mse_mbgdr)
print("R2 Score:", r2_mbgdr)

MSE: 2.122180837764743
R2 Score: 0.9993544770169708
